## Домашняя работа
ФИО: Карабанов Павел Александрович

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`



In [66]:
from threading import Lock
class ThreadSafeSingleton(type):
    _instances = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
         if cls not in cls._instances:
            # Захватываем блокировку только при первом создании
            with cls._lock:
                # ВТОРАЯ проверка (с блокировкой) - БЕЗОПАСНАЯ
                if cls not in cls._instances:
                    cls._instances[cls] = super().__call__(*args, **kwargs)
         return cls._instances[cls]

In [67]:
class DatabasePool(int, metaclass=ThreadSafeSingleton):
    def __init__(self):
        self.connection = 'connected'
        self.thread_number = 0

    def get_connection(self):
        if self.connection == 'connected':
            self.connection = 'disconnected'
        else: self.connection = 'connected'
        return self.connection



### проверка задания 1

In [70]:
type(DatabasePool())
# Создайте 3 экземпляра DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

# Убедитесь, что это один и тот же объект
assert pool1 is pool2 is pool3

#Проверьте, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
conn2 = pool2.get_connection()
assert conn1 != conn2
print((pool1 is pool2 is pool3),(conn1 != conn2))

True True


## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

In [12]:
class ThreadCounter(type):
    _instances = {}
    _counters = {}

    def __call__(cls, *args, **kwargs):

        if cls not in cls._instances:
            cls._counters[cls] = 0
        cls._counters[cls] += 1

        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

    def get_count(cls):
        return ThreadCounter._counters.get(cls, 0)


In [13]:
class User(metaclass=ThreadCounter):
    def __init__(self, name):
        self.name = name
        self.counter = ThreadCounter.get_count(self)

    def get_total_count(cls):
        return ThreadCounter.get_count(cls)

In [14]:
class Product(metaclass=ThreadCounter):
    def __init__(self, name):
        self.name = name
        self.counter = ThreadCounter.get_count(self)

    def get_total_count(cls):
        return ThreadCounter.get_count(cls)

### проверка задания 2

In [15]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

In [43]:
class MetaName(type):
    def __new__(cls, name, bases, attrs):
        # Добавляем метод describe в атрибуты класса
        def describe(cls):
            return f"Это объект класса {name}"

        attrs['describe'] = describe
        return super().__new__(cls, name, bases, attrs)


In [55]:
class Car(metaclass=MetaName):
    def __init__(self, name):
        self.name = name


class Book(metaclass=MetaName):
    def __init__(self, name):
        self.name = name


### проверка задания 3

In [56]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

In [80]:
class MetaSave(type):
    def __new__(cls, name, bases, attrs):
        # Добавляем метод describe в атрибуты класса
        if 'save' not in attrs:
            raise TypeError(f"Класс {name} должен содержать метод save")

        return super().__new__(cls, name, bases, attrs)




In [75]:
class User(metaclass=MetaSave):
    def __init__(self, name):
        self.name = name

    def save(self):
        pass

### проверка задания 4

In [81]:
# Проверка
user = User("Alice")
user.save()  # Должно работать

# Этот код должен вызвать ошибку:
class Message(metaclass=MetaSave):
    def __init__(self, text):
        self.text = text

TypeError: Класс Message должен содержать метод save